In [50]:
import json
import time

import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import  bs4 
import pandas as pd

In [48]:
base_url = "https://www.reddit.com/best/communities/%i/"
top_subs = dict()

for i in tqdm(range(1, 1267+1)):
    res = requests.get(base_url % i)
    soup = BeautifulSoup(res.content, 'html.parser')
    for container in soup.find_all("div", class_="community-list"):
        for child in container.children:
            if not isinstance(child, bs4.element.Tag): continue
            attrs = child.attrs.copy()
            attrs.update(dict((zip(['rank', 'category', 'members'], [''.join(t.strings).strip() for t in child.find_all('h6')]))))
            del attrs['members']
            del attrs['class']
            del attrs['id']
            top_subs[attrs.pop('data-community-id')]=attrs
    time.sleep(1)

100%|██████████| 1267/1267 [1:24:23<00:00,  4.00s/it]


In [49]:
len(top_subs)

316558

In [52]:
with open('../../data/interim/subreddit_categories.json', 'w+') as f:
    json.dump(top_subs, f)

In [58]:

df = pd.DataFrame(top_subs).T
df.to_csv('../../data/interim/subreddit_categories.csv')

In [67]:
df.groupby('category').size().sort_values(ascending=False).head(100).to_csv('../../data/interim/subreddit_category_counts.csv')

In [68]:
df.groupby('category').size().sort_values(ascending=False).head(100)

category
Gaming                        13584
Music                          5229
Technology                     4520
Internet Culture and Memes     4020
Place                          2832
                              ...  
?                                 5
Coming soon                       5
le                                5
~                                 5
bruh                              5
Length: 100, dtype: int64

In [63]:
interesting_categoreis = {"Mature Themes and Adult Content",
 "Meta/Reddit",
 "Culture, Race, and Ethnicity",
 "Gender",
 "Sexual Orientation",
 "Women's Health",
 "Beauty and Makeup",
 "Men's Health", }


In [66]:
df[df.category.isin(interesting_categoreis)][['data-prefixed-name',  'category']]

,data-prefixed-name,category
t5_2qzb6,r/IAmA,Meta/Reddit
t5_2to41,r/tifu,Meta/Reddit
t5_2qh3v,r/bestof,Meta/Reddit
t5_2rww2,r/MakeupAddiction,Beauty and Makeup
t5_2w844,r/NoStupidQuestions,Meta/Reddit
...,...,...
t5_8dn835,r/NEDPhilanders,Mature Themes and Adult Content
t5_algzks,r/RDXPro,Mature Themes and Adult Content
t5_59y609,r/WeAreAllTamils,"Culture, Race, and Ethnicity"
t5_48idty,r/Nepricans,Mature Themes and Adult Content
